# 04. Widgets & Visualizations

This module shows how to work with Jupyter Widgets with anywidget. It also shows how to create a Vitessce visualization of Portal data.

## Introduction to anywidgets

[Jupyter Widgets](https://ipywidgets.readthedocs.io/en/stable/) are interactive browser controls for notebooks. To solve the issue of multiple platforms with multiple ways of discovering front-end code and loading and executing JavaScript, [anywidget](https://anywidget.dev) was created.

In [ ]:
# !pip install traitlets vitessce[all]==3.3.1

In [ ]:
import traitlets
import anywidget

from vitessce import VitessceChainableConfig, VitessceConfigDatasetFile

In an anywidget, JavaScript can be added with the __esm_. Let's create a simple button.

In [ ]:
class CounterWidget(anywidget.AnyWidget):
    _esm = """
    function render({ model, el }) {
      let btn = document.createElement("button");
      btn.innerHTML = `count is ?`;
      el.appendChild(btn);
    }
    export default { render };
    """

counter = CounterWidget()
counter

This creates a button, but it does not yet have any functionality. We can create a value with a traitlet, which we can then refer to. If we add a click event listener, we can then increase the value of the button when we click on it. 

In [ ]:
# this example comes from https://anywidget.dev
class CounterWidget(anywidget.AnyWidget):
    _esm = """
    function render({ model, el }) {
      let count = () => model.get("value");
      let btn = document.createElement("button");
      btn.innerHTML = `count is ${count()}`;
      btn.addEventListener("click", () => {
        model.set("value", count() + 1);
        model.save_changes();
      });
      model.on("change:value", () => {
        btn.innerHTML = `count is ${count()}`;
      });
      el.appendChild(btn);
    }
    export default { render };
    """
    value = traitlets.Int(0).tag(sync=True)

counter = CounterWidget()
counter

We can then get the actual value of the button. Run the cell below. Then click on the button a few more times, and run the cell below again. See how it updates?

In [ ]:
counter.value

## Introduction to Vitessce

We can use these widgets to embed interactive visualizations. [Vitessce](http://vitessce.io) is a tool for exploration of spatial single cell data. Vitessce uses a config to create a visualization. We show an example below.

In [ ]:
conf = VitessceChainableConfig(
    schema_version="1.0.15", name="reg001_expr", description=""
).add_dataset(
    uid="A",
    name="SPRM",
    files=[
        VitessceConfigDatasetFile(
            file_type="anndata.zarr",
            url="https://assets.hubmapconsortium.org/69c70762689b20308bb049ac49653342/anndata-zarr/reg001_expr-anndata.zarr",
            options={
                "obsLocations": {"path": "obsm/xy"},
                "obsEmbedding": [
                    {"path": "obsm/tsne", "dims": [0, 1], "embeddingType": "t-SNE"}
                ],
                "obsSets": [
                    {
                        "name": "Cell K-Means [tSNE_All_Features]",
                        "path": "obs/Cell K-Means [tSNE_All_Features]",
                    },
                    {
                        "name": "Cell K-Means [Mean-All-SubRegions] Expression",
                        "path": "obs/Cell K-Means [Mean-All-SubRegions] Expression",
                    },
                    {
                        "name": "Cell K-Means [Mean] Expression",
                        "path": "obs/Cell K-Means [Mean] Expression",
                    },
                    {
                        "name": "Cell K-Means [Shape-Vectors]",
                        "path": "obs/Cell K-Means [Shape-Vectors]",
                    },
                    {
                        "name": "Cell K-Means [Texture]",
                        "path": "obs/Cell K-Means [Texture]",
                    },
                    {
                        "name": "Cell K-Means [Total] Expression",
                        "path": "obs/Cell K-Means [Total] Expression",
                    },
                    {
                        "name": "Cell K-Means [Covariance] Expression",
                        "path": "obs/Cell K-Means [Covariance] Expression",
                    },
                ],
                "obsFeatureMatrix": {"path": "X"},
            },
        ),
        VitessceConfigDatasetFile(
            file_type="raster.json",
            options={
                "schemaVersion": "0.0.2",
                "usePhysicalSizeScaling": False,
                "images": [
                    {
                        "name": "reg001_expr",
                        "type": "ome-tiff",
                        "url": "https://assets.hubmapconsortium.org/69c70762689b20308bb049ac49653342/ometiff-pyramids/pipeline_output/expr/reg001_expr.ome.tif",
                        "metadata": {
                            "omeTiffOffsetsUrl": "https://assets.hubmapconsortium.org/69c70762689b20308bb049ac49653342/output_offsets/pipeline_output/expr/reg001_expr.offsets.json",
                            "isBitmask": False,
                        },
                    },
                    {
                        "name": "reg001_mask",
                        "type": "ome-tiff",
                        "url": "https://assets.hubmapconsortium.org/69c70762689b20308bb049ac49653342/ometiff-pyramids/pipeline_output/mask/reg001_mask.ome.tif",
                        "metadata": {
                            "omeTiffOffsetsUrl": "https://assets.hubmapconsortium.org/69c70762689b20308bb049ac49653342/output_offsets/pipeline_output/mask/reg001_mask.offsets.json",
                            "isBitmask": True,
                        },
                    },
                ],
                "renderLayers": ["reg001_expr", "reg001_mask"],
            },
        ),
    ],
).set_coordination_value(
    c_type="embeddingType", c_scope="A", c_value="t-SNE"
).add_view(
    dataset_uid="A", component="description", x=0, y=8, w=3, h=4
).add_view(
    dataset_uid="A", component="layerController", x=0, y=0, w=3, h=8
).add_view(
    dataset_uid="A", component="spatial", x=3, y=0, w=4, h=8
).add_view(
    dataset_uid="A",
    component="scatterplot",
    x=7,
    y=0,
    w=3,
    h=8,
    coordination_scopes={"embeddingType": "A"},
).add_view(
    dataset_uid="A", component="obsSets", x=10, y=5, w=2, h=7
).add_view(
    dataset_uid="A",
    component="featureList",
    x=10,
    y=0,
    w=2,
    h=5,
    props={"variablesLabelOverride": "antigen"},
).add_view(
    dataset_uid="A",
    component="heatmap",
    x=3,
    y=8,
    w=7,
    h=4,
    props={"variablesLabelOverride": "antigen", "transpose": True},
)

conf.widget()

You can find more examples and information in this [Vitessce tutorial](https://github.com/vitessce/vitessce-python-tutorial).